In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import calendar
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'bi-presto.serving.data.production.internal'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [14]:
morning_zone = pd.read_csv('delhi_morning.csv')
morning_zone_hexes = morning_zone['pickup_location_hex_8'].unique().tolist()
print(morning_zone_hexes)

['883da11abbfffff', '883da11ab5fffff', '883da11ab7fffff', '883da11ad7fffff', '883da118cbfffff', '883da11ad5fffff', '883da118dbfffff', '883da11aa5fffff', '883da11a95fffff', '883da11ab3fffff', '883da11aa9fffff', '883da11a87fffff', '883da11a9bfffff', '883da11ab9fffff', '883da11acdfffff', '883da1185bfffff', '883da118c9fffff', '883da11ab1fffff', '883da11853fffff', '883da11a91fffff', '883da11a83fffff', '883da11ac3fffff', '883da11ae1fffff', '883da11aadfffff', '883da11abdfffff', '883da11a99fffff', '883da11aabfffff', '883da11aa3fffff', '883da11aebfffff', '883da11a8bfffff', '883da11a8dfffff', '883da11ae7fffff', '883da11ac1fffff', '883da11a9dfffff', '883da11a85fffff', '883da11ae9fffff', '883da11a33fffff', '883da118ddfffff', '883da11a13fffff', '883da11ac5fffff', '883da11aa1fffff', '883da11851fffff', '883da1036dfffff', '883da11aa7fffff', '883da11ae3fffff', '883da11a93fffff', '883da1034dfffff', '883da118d1fffff', '883da118d7fffff', '883da11ae5fffff', '883da10369fffff', '883da11a81fffff', '883da11aed

In [28]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
pickup_location_hex_8, pickup_location_latitude, pickup_location_longitude, drop_location_hex_8, drop_location_latitude, drop_location_longitude,
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20230306'
and yyyymmdd <= '20230310'
and city_name in ('Delhi')
and service_obj_service_name = 'Link'
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [29]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason
0,6409580811dc5f2f891f8f40,0.857,883da1a939fffff,28.627636,77.374275,Delhi,883da106c9fffff,28.606983,883da10619fffff,28.599156,77.349524,883da1a939fffff,28.627636,77.374275,77.372783,092240,09,0915,Link,09:00,4,20230309,dropped,None
1,6409582876454e7fc02a0cea,2.304,883da106cbfffff,28.606838,77.384018,Delhi,883da10689fffff,28.595783,883da106d5fffff,28.598350,77.396806,883da106cbfffff,28.606838,77.384018,77.388794,092312,09,0915,Link,09:00,4,20230309,dropped,None
2,6409585e9abbe025bb901013,NaN,883da11163fffff,28.497772,77.013163,Delhi,883da1112dfffff,28.488268,883da11129fffff,28.490963,77.012093,883da11abdfffff,28.448816,77.068588,77.011215,092406,09,0915,Link,09:00,4,20230309,expired,None
3,64095921721e576f836f8895,2.448,883da13365fffff,28.579052,77.440514,Delhi,883da13365fffff,28.579021,883da13365fffff,28.579028,77.440491,883da10657fffff,28.593482,77.336783,77.440491,092721,09,0915,Link,09:00,4,20230309,customerCancelled,Taking longer than expected
4,6409594c3d612b1ceefd7a17,1.472,883da11aa5fffff,28.433918,77.051674,Delhi,883da11ac1fffff,28.470150,883da11ac1fffff,28.470029,77.042944,883da11aa5fffff,28.433918,77.051674,77.042953,092804,09,0915,Link,09:00,4,20230309,dropped,None


In [30]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count
0,6409580811dc5f2f891f8f40,0.857,883da1a939fffff,28.627636,77.374275,Delhi,883da106c9fffff,28.606983,883da10619fffff,28.599156,77.349524,883da1a939fffff,28.627636,77.374275,77.372783,092240,09,0915,Link,09:00,4,20230309,dropped,None,40,22,09,1
1,6409582876454e7fc02a0cea,2.304,883da106cbfffff,28.606838,77.384018,Delhi,883da10689fffff,28.595783,883da106d5fffff,28.598350,77.396806,883da106cbfffff,28.606838,77.384018,77.388794,092312,09,0915,Link,09:00,4,20230309,dropped,None,12,23,09,1
2,6409585e9abbe025bb901013,NaN,883da11163fffff,28.497772,77.013163,Delhi,883da1112dfffff,28.488268,883da11129fffff,28.490963,77.012093,883da11abdfffff,28.448816,77.068588,77.011215,092406,09,0915,Link,09:00,4,20230309,expired,None,06,24,09,1
3,64095921721e576f836f8895,2.448,883da13365fffff,28.579052,77.440514,Delhi,883da13365fffff,28.579021,883da13365fffff,28.579028,77.440491,883da10657fffff,28.593482,77.336783,77.440491,092721,09,0915,Link,09:00,4,20230309,customerCancelled,Taking longer than expected,21,27,09,1
4,6409594c3d612b1ceefd7a17,1.472,883da11aa5fffff,28.433918,77.051674,Delhi,883da11ac1fffff,28.470150,883da11ac1fffff,28.470029,77.042944,883da11aa5fffff,28.433918,77.051674,77.042953,092804,09,0915,Link,09:00,4,20230309,dropped,None,04,28,09,1


In [31]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Delhi')
and resolution = 8
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [32]:
cluster_mapping = df_cluster_copy.copy()
cluster_mapping = cluster_mapping.rename(columns = {'hex_id':'pickup_location_hex_8'})
df = df.merge(cluster_mapping[['pickup_location_hex_8', 'cluster']], on = 'pickup_location_hex_8', how = 'left')
cluster_mapping = cluster_mapping.rename(columns = {'pickup_location_hex_8':'drop_location_hex_8'})
df = df.rename(columns = {'cluster':'pickup_cluster'})
df = df.merge(cluster_mapping[['drop_location_hex_8', 'cluster']], on = 'drop_location_hex_8', how = 'left')
df = df.rename(columns = {'cluster':'drop_cluster'})
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster
0,6409580811dc5f2f891f8f40,0.857,883da1a939fffff,28.627636,77.374275,Delhi,883da106c9fffff,28.606983,883da10619fffff,28.599156,77.349524,883da1a939fffff,28.627636,77.374275,77.372783,092240,09,0915,Link,09:00,4,20230309,dropped,None,40,22,09,1,NCR N sec 2262,NCR N sec 2262
1,6409582876454e7fc02a0cea,2.304,883da106cbfffff,28.606838,77.384018,Delhi,883da10689fffff,28.595783,883da106d5fffff,28.598350,77.396806,883da106cbfffff,28.606838,77.384018,77.388794,092312,09,0915,Link,09:00,4,20230309,dropped,None,12,23,09,1,NCR N Sec 121,NCR N sec 6354
2,6409585e9abbe025bb901013,NaN,883da11163fffff,28.497772,77.013163,Delhi,883da1112dfffff,28.488268,883da11129fffff,28.490963,77.012093,883da11abdfffff,28.448816,77.068588,77.011215,092406,09,0915,Link,09:00,4,20230309,expired,None,06,24,09,1,Palam Vihar 3,Sector 45-46
3,64095921721e576f836f8895,2.448,883da13365fffff,28.579052,77.440514,Delhi,883da13365fffff,28.579021,883da13365fffff,28.579028,77.440491,883da10657fffff,28.593482,77.336783,77.440491,092721,09,0915,Link,09:00,4,20230309,customerCancelled,Taking longer than expected,21,27,09,1,NCR G Kaushambi Metro,NCR N sec 2262
4,6409594c3d612b1ceefd7a17,1.472,883da11aa5fffff,28.433918,77.051674,Delhi,883da11ac1fffff,28.470150,883da11ac1fffff,28.470029,77.042944,883da11aa5fffff,28.433918,77.051674,77.042953,092804,09,0915,Link,09:00,4,20230309,dropped,None,04,28,09,1,G Patel Nagar,Sector 45-46


In [33]:
cluster_grouped_morning = df[df['hour'].isin(['08', '09', '10', '11'])].groupby(['pickup_cluster'])['order_id'].nunique().reset_index()
cluster_grouped_evening = df[df['hour'].isin(['17', '18', '19', '20', '21'])].groupby(['pickup_cluster'])['order_id'].nunique().reset_index()
cluster_grouped_morning = cluster_grouped_morning.sort_values(by = 'order_id', ascending=False)
cluster_grouped_evening = cluster_grouped_evening.sort_values(by = 'order_id', ascending=False)

top_clusters_morning = cluster_grouped_morning.head(40)
top_clusters_morning['orders_per_day'] = top_clusters_morning['order_id'].apply(lambda x: x/5)
top_clusters_morning['orders_per_hour'] = top_clusters_morning['orders_per_day'].apply(lambda x: x/4)
top_clusters_morning['orders_per_20_sec_batch'] = top_clusters_morning['orders_per_hour'].apply(lambda x: x/180)

top_clusters_evening = cluster_grouped_evening.head(40)
top_clusters_evening['orders_per_day'] = top_clusters_evening['order_id'].apply(lambda x: x/5)
top_clusters_evening['orders_per_hour'] = top_clusters_evening['orders_per_day'].apply(lambda x: x/4)
top_clusters_evening['orders_per_20_sec_batch'] = top_clusters_evening['orders_per_hour'].apply(lambda x: x/180)
top_clusters_morning

,pickup_cluster,order_id,orders_per_day,orders_per_hour,orders_per_20_sec_batch
186,NCR N sec 6354,5307,1061.4,265.35,1.474167
160,NCR G Vijay Nagar 2,3942,788.4,197.10,1.095000
246,Sec-55,3871,774.2,193.55,1.075278
162,NCR N Gharoli,3744,748.8,187.20,1.040000
247,Sec-57,3433,686.6,171.65,0.953611
185,NCR N sec 2262,3420,684.0,171.00,0.950000
248,Sector 45-46,2999,599.8,149.95,0.833056
244,Sec-39,2980,596.0,149.00,0.827778
35,DLF Phase-1,2845,569.0,142.25,0.790278
182,NCR N WaveCity Center,2831,566.2,141.55,0.786389


In [35]:
#Filter the df on morning and evening peak hours
df_morning = df[df['hour'].isin(['08', '09', '10', '11'])]
df_evening = df[df['hour'].isin(['17', '18', '19', '20', '21'])]

#Groupby on hex and quarter hour to get the number of orders coming in every hex
df_morning_grouped = df_morning.groupby(['yyyymmdd', 'hour', 'pickup_location_hex_8'])['order_id'].nunique().reset_index()
df_evening_grouped = df_evening.groupby(['yyyymmdd', 'hour', 'pickup_location_hex_8'])['order_id'].nunique().reset_index()
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped.head()

,yyyymmdd,hour,pickup_location_hex_8,num_of_orders
0,20230306,17,883da10003fffff,1
1,20230306,17,883da1000dfffff,1
2,20230306,17,883da10029fffff,2
3,20230306,17,883da1002dfffff,1
4,20230306,17,883da10043fffff,5


In [36]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'hour', 'pickup_location_hex_8'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'hour', 'pickup_location_hex_8'], how = 'left')

In [37]:
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])

In [40]:
#Query data for mismatch

q_online_captains = """
select * from pricing.cluster_temporal_hotness
where service_detail_id = '5da4660028af187d8d52cc3c'
"""


#Load data into pandas table
df_mismatch_copy = pd.read_sql(q_online_captains, connection)

In [46]:
#View the dataset
df_mismatch = df_mismatch_copy.copy()
df_mismatch.head()

,hotness_category,pickupclusters,week_period,time_period,temporal_id,total_footfall,total_revenue,hex_count,total_revenue_normalized,normalized_revenue_contribution_percentage,cum_revenue_contribution_percentage,revenue_contribution_percentile_rank,temporal_stress,temporal_mismatch,spatial_stress,spatial_mismatch,cluster_hotness_score,run_date,strategy_version,service_detail_id
0,cold,ALIGANJ GAYA,Sunday,1600,Sunday-1600,0,0.0,0,0.0,0.0,0.0,0.0,0.529320,0.529320,0.0,0.0,0.264660,20230307,HOT_COLD_3_0,5da4660028af187d8d52cc3c
1,cold,ALIGANJ GAYA,Sunday,0200,Sunday-0200,0,0.0,0,0.0,0.0,0.0,0.0,0.575125,0.575125,0.0,0.0,0.287562,20230307,HOT_COLD_3_0,5da4660028af187d8d52cc3c
2,super-cold,ALIGANJ GAYA,Sunday,0800,Sunday-0800,0,0.0,0,0.0,0.0,0.0,0.0,0.275737,0.275737,0.0,0.0,0.137869,20230307,HOT_COLD_3_0,5da4660028af187d8d52cc3c
3,cold,ALIGANJ GAYA,Sunday,0300,Sunday-0300,0,0.0,0,0.0,0.0,0.0,0.0,0.537774,0.537774,0.0,0.0,0.268887,20230307,HOT_COLD_3_0,5da4660028af187d8d52cc3c
4,cold,ALIGANJ GAYA,Sunday,0400,Sunday-0400,0,0.0,0,0.0,0.0,0.0,0.0,0.453353,0.453353,0.0,0.0,0.226677,20230307,HOT_COLD_3_0,5da4660028af187d8d52cc3c


In [48]:
#rename column
df_mismatch = df_mismatch.rename(columns = {'week_period':'weekday_name', 'pickupclusters':'pickup_cluster'})
df_mismatch['hour'] = df_mismatch['time_period'].apply(lambda x: x[:2])
df_mismatch.head()

,hotness_category,pickup_cluster,weekday_name,time_period,temporal_id,total_footfall,total_revenue,hex_count,total_revenue_normalized,normalized_revenue_contribution_percentage,cum_revenue_contribution_percentage,revenue_contribution_percentile_rank,temporal_stress,temporal_mismatch,spatial_stress,spatial_mismatch,cluster_hotness_score,run_date,strategy_version,service_detail_id,hour
0,cold,ALIGANJ GAYA,Sunday,1600,Sunday-1600,0,0.0,0,0.0,0.0,0.0,0.0,0.529320,0.529320,0.0,0.0,0.264660,20230307,HOT_COLD_3_0,5da4660028af187d8d52cc3c,16
1,cold,ALIGANJ GAYA,Sunday,0200,Sunday-0200,0,0.0,0,0.0,0.0,0.0,0.0,0.575125,0.575125,0.0,0.0,0.287562,20230307,HOT_COLD_3_0,5da4660028af187d8d52cc3c,02
2,super-cold,ALIGANJ GAYA,Sunday,0800,Sunday-0800,0,0.0,0,0.0,0.0,0.0,0.0,0.275737,0.275737,0.0,0.0,0.137869,20230307,HOT_COLD_3_0,5da4660028af187d8d52cc3c,08
3,cold,ALIGANJ GAYA,Sunday,0300,Sunday-0300,0,0.0,0,0.0,0.0,0.0,0.0,0.537774,0.537774,0.0,0.0,0.268887,20230307,HOT_COLD_3_0,5da4660028af187d8d52cc3c,03
4,cold,ALIGANJ GAYA,Sunday,0400,Sunday-0400,0,0.0,0,0.0,0.0,0.0,0.0,0.453353,0.453353,0.0,0.0,0.226677,20230307,HOT_COLD_3_0,5da4660028af187d8d52cc3c,04


In [53]:
df_mismatch.groupby(['hotness_category'])['cluster_hotness_score'].median()

hotness_category
cold          0.234040
hot           0.753277
medium        0.463538
super-cold    0.121254
super-hot     0.885640
Name: cluster_hotness_score, dtype: float64

In [49]:
df_morning = df_morning.merge(df_mismatch[['weekday_name', 'hour', 'pickup_cluster', 'cluster_hotness_score']],
                               on = ['weekday_name', 'hour', 'pickup_cluster'], how = 'left')
df_evening = df_evening.merge(df_mismatch[['weekday_name', 'hour', 'pickup_cluster', 'cluster_hotness_score']],
                               on = ['weekday_name', 'hour', 'pickup_cluster'], how = 'left')
df_evening.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster,num_of_orders,weekday_name,cluster_hotness_score
0,6405e8131215c13df1c8314c,1.564,883da106c1fffff,28.597612,77.377388,Delhi,883da1a931fffff,28.620621,883da1a931fffff,28.620530,77.378556,883da106c1fffff,28.597612,77.377388,77.378609,184811,18,1845,Link,18:59,1,20230306,dropped,None,11,48,06,1,NCR N sec 6354,NCR N sec 6354,118,Monday,0.469124
1,6405e8131215c13df1c8314c,1.564,883da106c1fffff,28.597612,77.377388,Delhi,883da1a931fffff,28.620621,883da1a931fffff,28.620530,77.378556,883da106c1fffff,28.597612,77.377388,77.378609,184811,18,1845,Link,18:59,1,20230306,dropped,None,11,48,06,1,NCR N sec 6354,NCR N sec 6354,118,Monday,0.652811
2,6405e8131215c13df1c8314c,1.564,883da106c1fffff,28.597612,77.377388,Delhi,883da1a931fffff,28.620621,883da1a931fffff,28.620530,77.378556,883da106c1fffff,28.597612,77.377388,77.378609,184811,18,1845,Link,18:59,1,20230306,dropped,None,11,48,06,1,NCR N sec 6354,NCR N sec 6354,118,Monday,0.359327
3,6405e87801b8b86aa8bd5952,NaN,883da11081fffff,28.563860,77.122528,Delhi,883da11081fffff,28.563860,883da11081fffff,28.563663,77.122411,883da1176dfffff,28.610507,77.114565,77.122528,184952,18,1845,Link,18:59,1,20230306,customerCancelled,Order cancelled before rider was mapped,52,49,06,1,Mehram Nagar_East,Delhi_Central,11,Monday,0.434120
4,6405e87801b8b86aa8bd5952,NaN,883da11081fffff,28.563860,77.122528,Delhi,883da11081fffff,28.563860,883da11081fffff,28.563663,77.122411,883da1176dfffff,28.610507,77.114565,77.122528,184952,18,1845,Link,18:59,1,20230306,customerCancelled,Order cancelled before rider was mapped,52,49,06,1,Mehram Nagar_East,Delhi_Central,11,Monday,0.461054


In [51]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'weekday_name', 'cluster_hotness_score']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'weekday_name', 'cluster_hotness_score']]

In [52]:
df_evening_filter.to_csv('delhi_demand_supply_evening_vw2.csv', index = False)
df_morning_filter.to_csv('delhi_demand_supply_morning_vw2.csv', index = False)

In [6]:
#Query data for online captains

q_online_captains = """
select
        yyyymmdd, substr(hhmm, 1, 2) as hour,
        count(distinct captain_id) as Online_Captains
from 
(select 
        a.*, city, cluster
        
from 
        hive.datasets.supplycursory_history a 
        left join datasets.city_cluster_hex b 
        on a.location = b.hex_id and a.resolution = cast(b.resolution as varchar)
where 
        yyyymmdd >= '20230313'
        and yyyymmdd <='20230316'
        and city in ('Delhi')
        and a.resolution = '8' 
        and service in ('Link')
        and substr(hhmm, 1, 2) in ('08', '09', '10', '11')
        and location in ('883da11ad3fffff', '883da111abfffff', '883da11addfffff', '883da111edfffff', '883da111e9fffff', '883da111e5fffff', '883da10349fffff',
 '883da111c7fffff', '883da11ad5fffff', '883da1034dfffff', '883da111a7fffff', '883da11ad7fffff', '883da111a3fffff', '883da11a99fffff', '883da111a5fffff',
   '883da111ebfffff', '883da11a9bfffff', '883da111a1fffff', '883da11ad1fffff', '883da1118dfffff', '883da111b5fffff', '883da11185fffff', '883da111a9fffff',
     '883da11adbfffff', '883da111bdfffff', '883da1118bfffff', '883da10341fffff', '883da111e1fffff', '883da11ad9fffff', '883da111e3fffff', '883da111adfffff',
       '883da11189fffff', '883da11187fffff', '883da11181fffff', '883da111c5fffff', '883da111e7fffff', '883da111b9fffff', '883da11183fffff')
)
group by 1,2
"""

#Load data into pandas table
df_online_captains_copy = pd.read_sql(q_online_captains, connection)

In [7]:
#Make a copy of the dataset and view it
df_online_captains = df_online_captains_copy.copy()
df_online_captains.sort_values(by = 'hour')

,yyyymmdd,hour,Online_Captains
0,20230313,08,844
2,20230315,08,943
7,20230316,08,914
8,20230314,08,869
4,20230313,09,1253
9,20230315,09,1331
12,20230316,09,1395
13,20230314,09,1306
3,20230315,10,1662
10,20230316,10,1716


In [5]:
df_online_captains.to_csv('gurgaon_zone_online_captains.csv', index = False)

In [24]:
#Rename cluster column
df_online_captains = df_online_captains.rename(columns = {'cluster':'pickup_cluster'})
df_online_captains.head()

,yyyymmdd,hour,pickup_cluster,Online_Captains
0,20230307,09,Sec-39,196
1,20230306,10,M MG metro,161
2,20230309,09,Sec-57,67
3,20230306,08,Sushant lok,27
4,20230307,11,Sec-55,177


In [25]:
#Merge online captains with demand data
df_morning_grouped = df_morning_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_evening_grouped = df_evening_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_morning_grouped['demand_to_supply_ratio'] = round(df_morning_grouped['num_of_orders']/df_morning_grouped['Online_Captains'], 4)
df_evening_grouped['demand_to_supply_ratio'] = round(df_evening_grouped['num_of_orders']/df_evening_grouped['Online_Captains'], 4)

df_morning_grouped = df_morning_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped = df_evening_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped.head()

,yyyymmdd,hour,pickup_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio
0,20230306,17,DLF City Phase 4,15,NaN,NaN
1,20230306,17,DLF Phase-1,99,NaN,NaN
2,20230306,17,G Patel Nagar,93,NaN,NaN
3,20230306,17,G Sadar Bazar,90,NaN,NaN
4,20230306,17,Gurugram Sec 44,45,NaN,NaN


In [27]:
df_morning_hour = df_morning_grouped.groupby(['yyyymmdd', 'hour'])['num_of_orders', 'Online_Captains'].sum().reset_index()
df_morning_hour['demand_to_supply'] = df_morning_hour['num_of_orders']/df_morning_hour['Online_Captains']
df_morning_hour

,yyyymmdd,hour,num_of_orders,Online_Captains,demand_to_supply
0,20230306,08,1584,2020,0.784158
1,20230306,09,3018,2909,1.037470
2,20230306,10,1929,3481,0.554151
3,20230306,11,1114,3599,0.309530
4,20230307,08,1251,1726,0.724797
5,20230307,09,2635,2486,1.059936
6,20230307,10,1454,2997,0.485152
7,20230307,11,862,3066,0.281148
8,20230308,08,308,734,0.419619
9,20230308,09,318,869,0.365938


In [16]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio
0,6409568576454e7fc02a0948,NaN,,NaN,NaN,Delhi,883da11107fffff,28.502649,883da11107fffff,28.502695,77.032671,883da11a39fffff,28.451952,77.012890,77.032683,091613,09,0915,Link,09:00,4,20230309,expired,None,13,16,09,1,Dharam_Colony,Sec-10A,114.0,26.0,4.3846
1,640956b1b122592c61e0e119,NaN,883da106e7fffff,28.571661,77.382301,Delhi,883da106a9fffff,28.571867,883da106e7fffff,28.571710,77.382351,883da10589fffff,28.496311,77.401838,77.384453,091657,09,0915,Link,09:00,4,20230309,customerCancelled,order cancelled before rider accepted,57,16,09,1,NCR N Sec 754839,NCR N Sec 142,190.0,134.0,1.4179
2,640956ba70d3b7767d091759,1.251,883da13acbfffff,28.520071,77.531219,Delhi,883da13acbfffff,28.520052,883da13acbfffff,28.520067,77.531235,883da10655fffff,28.594005,77.324569,77.531189,091706,09,0915,Link,09:00,4,20230309,customerCancelled,Taking longer than expected,06,17,09,1,NCR N Suraj Pur,NCR N Gharoli,23.0,73.0,0.3151
3,640956f4a783dd64cec2b3a0,1.897,883da10629fffff,28.569284,77.316963,Delhi,883da10451fffff,28.557844,883da10451fffff,28.557920,77.371983,883da10629fffff,28.569284,77.316963,77.371941,091804,09,0915,Link,09:00,4,20230309,dropped,None,04,18,09,1,NCR N Sec 104,NCR N FilmCity,141.0,204.0,0.6912
4,640956f7f6b616544577d8c2,0.080,883da1384bfffff,28.469109,77.496048,Delhi,883da13843fffff,28.457357,883da13843fffff,28.457379,77.500463,883da1384bfffff,28.469109,77.496048,77.500458,091807,09,0915,Link,09:00,4,20230309,dropped,None,07,18,09,1,NCR N Venice Mall,NCR N Sharda University 2,99.0,124.0,0.7984


In [17]:
#Create a column for weekdays
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio,weekday_name
0,6409568576454e7fc02a0948,NaN,,NaN,NaN,Delhi,883da11107fffff,28.502649,883da11107fffff,28.502695,77.032671,883da11a39fffff,28.451952,77.012890,77.032683,091613,09,0915,Link,09:00,4,20230309,expired,None,13,16,09,1,Dharam_Colony,Sec-10A,114.0,26.0,4.3846,Thursday
1,640956b1b122592c61e0e119,NaN,883da106e7fffff,28.571661,77.382301,Delhi,883da106a9fffff,28.571867,883da106e7fffff,28.571710,77.382351,883da10589fffff,28.496311,77.401838,77.384453,091657,09,0915,Link,09:00,4,20230309,customerCancelled,order cancelled before rider accepted,57,16,09,1,NCR N Sec 754839,NCR N Sec 142,190.0,134.0,1.4179,Thursday
2,640956ba70d3b7767d091759,1.251,883da13acbfffff,28.520071,77.531219,Delhi,883da13acbfffff,28.520052,883da13acbfffff,28.520067,77.531235,883da10655fffff,28.594005,77.324569,77.531189,091706,09,0915,Link,09:00,4,20230309,customerCancelled,Taking longer than expected,06,17,09,1,NCR N Suraj Pur,NCR N Gharoli,23.0,73.0,0.3151,Thursday
3,640956f4a783dd64cec2b3a0,1.897,883da10629fffff,28.569284,77.316963,Delhi,883da10451fffff,28.557844,883da10451fffff,28.557920,77.371983,883da10629fffff,28.569284,77.316963,77.371941,091804,09,0915,Link,09:00,4,20230309,dropped,None,04,18,09,1,NCR N Sec 104,NCR N FilmCity,141.0,204.0,0.6912,Thursday
4,640956f7f6b616544577d8c2,0.080,883da1384bfffff,28.469109,77.496048,Delhi,883da13843fffff,28.457357,883da13843fffff,28.457379,77.500463,883da1384bfffff,28.469109,77.496048,77.500458,091807,09,0915,Link,09:00,4,20230309,dropped,None,07,18,09,1,NCR N Venice Mall,NCR N Sharda University 2,99.0,124.0,0.7984,Thursday


In [18]:
num_of_hex_morning = df_morning.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_evening = df_evening.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_morning = num_of_hex_morning.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})
num_of_hex_evening = num_of_hex_evening.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})

df_morning = df_morning.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_evening = df_evening.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_morning['online_captains_per_hex'] = round(df_morning['Online_Captains']/df_morning['num_of_hexes'], 4)
df_evening['online_captains_per_hex'] = round(df_evening['Online_Captains']/df_evening['num_of_hexes'], 4)

In [19]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]

In [20]:
df_evening_filter.to_csv('delhi_demand_supply_evening.csv', index = False)
df_morning_filter.to_csv('delhi_demand_supply_morning.csv', index = False)